In [1]:
!pip install bigframes -q


In [2]:
from google.cloud import bigquery

import bigframes.pandas as bpd
from bigframes.ml.linear_model import LinearRegression as BQLinearRegression

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression as SKLinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

import pandas as pd
import matplotlib.pyplot as plt
sns.set_theme(color_codes=True)

In [3]:
PROJECT_ID = "arctic-eye-446816-j7"
REGION = "US"

In [4]:
bpd.options.bigquery.project = PROJECT_ID
bpd.options.bigquery.location = REGION

In [5]:
df = bpd.read_gbq("bigquery-public-data.ml_datasets.penguins")
df.head()

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Gentoo penguin (Pygoscelis papua),Biscoe,50.5,15.9,225.0,5400.0,MALE
1,Gentoo penguin (Pygoscelis papua),Biscoe,45.1,14.5,215.0,5000.0,FEMALE
2,Adelie Penguin (Pygoscelis adeliae),Torgersen,41.4,18.5,202.0,3875.0,MALE
3,Adelie Penguin (Pygoscelis adeliae),Torgersen,38.6,17.0,188.0,2900.0,FEMALE
4,Gentoo penguin (Pygoscelis papua),Biscoe,46.5,14.8,217.0,5200.0,FEMALE


In [6]:
df.info()

<class 'bigframes.dataframe.DataFrame'>


Index: 344 entries, 0 to 343
Data columns (total 7 columns):


/opt/conda/lib/python3.10/site-packages/bigframes/core/blocks.py:125: NullIndexPreviewWarning: Creating object with Null Index. Null Index is a preview feature.
  warnings.warn(


  #  Column             Non-Null Count    Dtype
---  -----------------  ----------------  -------
  0  species            344 non-null      string
  1  island             344 non-null      string
  2  culmen_length_mm   342 non-null      Float64
  3  culmen_depth_mm    342 non-null      Float64
  4  flipper_length_mm  342 non-null      Float64
  5  body_mass_g        342 non-null      Float64
  6  sex                334 non-null      string
dtypes: Float64(4), string(3)


memory usage: 22016 bytes


In [8]:
df[df.sex.isnull()]

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
6,Adelie Penguin (Pygoscelis adeliae),Dream,37.5,18.9,179.0,2975.0,<NA>
23,Gentoo penguin (Pygoscelis papua),Biscoe,<NA>,<NA>,<NA>,<NA>,<NA>
40,Adelie Penguin (Pygoscelis adeliae),Torgersen,<NA>,<NA>,<NA>,<NA>,<NA>
136,Adelie Penguin (Pygoscelis adeliae),Torgersen,34.1,18.1,193.0,3475.0,<NA>
160,Gentoo penguin (Pygoscelis papua),Biscoe,44.5,14.3,216.0,4100.0,<NA>
205,Adelie Penguin (Pygoscelis adeliae),Torgersen,37.8,17.3,180.0,3700.0,<NA>
235,Gentoo penguin (Pygoscelis papua),Biscoe,47.3,13.8,216.0,4725.0,<NA>
244,Adelie Penguin (Pygoscelis adeliae),Torgersen,42.0,20.2,190.0,4250.0,<NA>
254,Adelie Penguin (Pygoscelis adeliae),Torgersen,37.8,17.1,186.0,3300.0,<NA>
332,Gentoo penguin (Pygoscelis papua),Biscoe,46.2,14.4,214.0,4650.0,<NA>


In [10]:
sns.scatterplot(x='flipper_length_mm', y = 'body_mass_g', data=df, hue='sex')
plt.show()
##https://matplotlib.org/stable/gallery/lines_bars_and_markers/scatter_with_legend.html#sphx-glr-gallery-lines-bars-and-markers-scatter-with-legend-py

TypeError: Data source must be a DataFrame or Mapping, not <class 'bigframes.dataframe.DataFrame'>.